This code follows the Keras introduction to sequence-to-sequence learning blog post which can be found here:

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [1]:
# Import dependencies
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

C:\Users\Marvin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Set hyperparameters/arguments
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'fra-eng/fra.txt'

In [3]:
# Read input texts and output texts from file into lists
input_texts = []
target_texts = []
input_chars = set()
target_chars = set() # Set faster than list to search through

with open(data_path, 'r', encoding='utf-8') as f:
    count = 0
    for line in f:
        if count == num_samples:
            break
        # Separate input and target pairs and add a start sequence character to target text
        input_text, target_text = line.split('\t')
        target_text = '\t' + target_text
        input_texts.append(input_text)
        target_texts.append(target_text)
        # Create set containing unique input characters
        for char in input_text:
            if char not in input_chars:
                input_chars.add(char)
        # Create set containing unique target characters
        for char in target_text:
            if char not in target_chars:
                target_chars.add(char)
        count += 1

In [4]:
# Convert the set of input characters and target characters to lists and sort them
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))

# Information on variables
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = len(max(input_texts, key=len))
max_decoder_seq_length = len(max(target_texts, key=len))

# Print information
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [5]:
# Create dictionaries that maps characters to indices and vice versa
input_token_index = dict((char, i) for i, char in enumerate(input_chars))
target_token_index = dict((char, i) for i, char in enumerate(target_chars))

reverse_input_token_index = dict((i, c) for i, c in enumerate(input_chars))
reverse_target_token_index = dict((i, c) for i, c in enumerate(target_chars))

In [6]:
# Use text to create arrays for input and output (target) sequences

# Initialize input and target data
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# Iterate through examples
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Iterate through input sequence
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    # Iterate through output sequence
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Print variable shapes
print('Encoder input shape:', encoder_input_data.shape)
print('Decoder input shape:', decoder_input_data.shape)
print('Decoder target shape:', decoder_target_data.shape)

Encoder input shape: (10000, 16, 71)
Decoder input shape: (10000, 59, 94)
Decoder target shape: (10000, 59, 94)


In [7]:
# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
_, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [8]:
# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Fit model
model.fit(x=[encoder_input_data, decoder_input_data], 
          y=decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2
         )

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.9174 - val_loss: 0.9425
Epoch 2/100
8000/8000 [==============================] - 24s 3ms/step - loss: 0.7221 - val_loss: 0.7848
Epoch 3/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.6175 - val_loss: 0.7114
Epoch 4/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.5614 - val_loss: 0.6512
Epoch 5/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.5211 - val_loss: 0.6227
Epoch 6/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.4881 - val_loss: 0.5969
Epoch 7/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.4625 - val_loss: 0.5725
Epoch 8/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.4399 - val_loss: 0.5537
Epoch 9/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.4206 - val_loss: 0.5418
Epoch 10/100
800

8000/8000 [==============================] - 20s 2ms/step - loss: 0.0728 - val_loss: 0.7047
Epoch 79/100
8000/8000 [==============================] - 20s 2ms/step - loss: 0.0719 - val_loss: 0.7098
Epoch 80/100
8000/8000 [==============================] - 20s 2ms/step - loss: 0.0701 - val_loss: 0.7162
Epoch 81/100
8000/8000 [==============================] - 20s 2ms/step - loss: 0.0692 - val_loss: 0.7249
Epoch 82/100
8000/8000 [==============================] - 20s 2ms/step - loss: 0.0681 - val_loss: 0.7240
Epoch 83/100
8000/8000 [==============================] - 19s 2ms/step - loss: 0.0671 - val_loss: 0.7233
Epoch 84/100
8000/8000 [==============================] - 20s 2ms/step - loss: 0.0655 - val_loss: 0.7344
Epoch 85/100
8000/8000 [==============================] - 20s 2ms/step - loss: 0.0650 - val_loss: 0.7357
Epoch 86/100
8000/8000 [==============================] - 20s 2ms/step - loss: 0.0640 - val_loss: 0.7370
Epoch 87/100
8000/8000 [==============================] - 20s 3ms/st

In [9]:
# Save model
model.save('s2s.h5')

C:\Users\Marvin\Anaconda3\lib\site-packages\keras\engine\topology.py:2364: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [10]:
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states) 

# Decoder states inputs
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder outputs
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

# Decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [11]:
"""
This function takes the previously calculated state vectors (hidden state & cell state) and output token to 
predict the next state vectors and output token. It does this iteratively until it predicts the "end sequence" 
token (or reaches a certain amount of iterations) to at which point it stops. Each predicted token will be 
converted to its character representation and added to a string which will hopefully be the correct translation
by the end of the loop.
"""

def decode_sequence(input_seq):
    # Initialize hidden and cell state to be the hidden and cell state from the encoder
    states_value = encoder_model.predict(input_seq) # [hidden state, cell state]

    # Initialize decoder input to the "start character"
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Initialize an empty string to hold the translation
    decoded_sentence = ''
    
    while True:
        # Predict the next output token and states
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Add the predicted tokens character representation to the string
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit loop if the "end sequence" character is predicted or if the decoded sentence reaches a max length
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            break

        # Update the decoder input with the new predicted character
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update the states
        states_value = [h, c]

    return decoded_sentence

In [13]:
# Prints predicted translations
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index: seq_index + 1] # shape (1, max_encoder_seq_length, num_encoder_tokens)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: I see.
Decoded sentence: Je l'ai sauvé.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: J'ai gagné !

-
Input sentence: I won!
Decoded senten